# Uvod v NumPy


## Understanding Data Types in Python



```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```


```python
# Python code
result = 0
for i in range(100):
    result += i
```


### A Python Integer Is More Than Just an Integer



```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```



<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/cint_vs_pyint.png" alt="Integer Memory Layout">

### A Python List Is More Than Just a List




<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/array_vs_list.png" alt="Array Memory Layout">

### Fixed-Type Arrays in Python



## How Vectorization Makes Code Faster


<p><img alt="Translating Python code to bytecode" src="https://s3.amazonaws.com/dq-content/289/bytecode.svg"></p>


<table>
<thead>
<tr>
<th>Language Type</th>
<th>Example</th>
<th>Time taken to write program</th>
<th>Control over program performance</th>
</tr>
</thead>
<tbody>
<tr>
<td>High-Level</td>
<td>Python</td>
<td>Low</td>
<td>Low</td>
</tr>
<tr>
<td>Low-Level</td>
<td>C</td>
<td>High</td>
<td>High</td>
</tr>
</tbody>
</table>



<p><img alt="For loop to sum rows" src="https://s3.amazonaws.com/dq-content/289/for_loop.svg"></p>

In [3]:
my_numbers = [[6,5], [1,3], [5,6]]

sums = []

for row in my_numbers:
    row_sum = row[0] + row[1]
    sums.append(row_sum)
    
print(sums)    

[11, 4, 11]




<p><img alt="Unvectorized operation" src="https://s3.amazonaws.com/dq-content/289/unvectorized.svg"></p>


<p><img alt="Vectorized operation" src="https://s3.amazonaws.com/dq-content/289/vectorized.svg"></p>



## Numpy library

[Dokumentacija](http://www.numpy.org/)

## Introduction to Ndarrays

<img alt="Dimensional Arrays" src="./images/one_dim.svg">

<img alt="Dimensional Arrays" src="./images/Two_Dim.svg">

## Priprava podatkov za delo

<div>

<ul>
<li><code>pickup_year</code>: The year of the trip.</li>
<li><code>pickup_month</code>: The month of the trip (January is <code>1</code>, December is <code>12</code>).</li>
<li><code>pickup_day</code>: The day of the month of the trip.</li>
<li><code>pickup_location_code</code>: The airport or <a target="_blank" href="https://en.wikipedia.org/wiki/Boroughs_of_New_York_City">borough</a> where the trip started.</li>
<li><code>dropoff_location_code</code>: The airport or borough where the trip finished.</li>
<li><code>trip_distance</code>: The distance of the trip in miles.</li>
<li><code>trip_length</code>: The length of the trip in seconds.</li>
<li><code>fare_amount</code>: The base fare of the trip, in dollars.</li>
<li><code>total_amount</code>: The total amount charged to the passenger, including all fees, tolls and tips.</li>
</ul>

</div>

    pickup_year,pickup_month,pickup_day,pickup_dayofweek
    2016,1,1,5
    2016,1,1,5
    2016,1,1,5
    2016,1,1,5

    # our list of lists is stored as data_list
    data_ndarray = np.array(data_list)

In [1]:
import csv
import numpy as np



## Array Shapes

<div class="alert alert-block alert-info">
<b>Vaja:</b> Assign the shape of taxi to taxi_shape. Print the result.</div>

## Selecting and Slicing Rows and Items from ndarrays

<img alt="Dimensional Arrays" src="./images/selection_rows.svg">

    ndarray[row_index,column_index]

    # or if you want to select all
    # columns for a given set of rows
    ndarray[row_index]

<img alt="Dimensional Arrays" src="./images/selection_item.svg">

<div class="alert alert-block alert-info">
<b>Vaja:</b> From the taxi ndarray:
- Select the row at index 0. Assign it to row_0.
- Select every column for the rows at indexes 391 to 500 inclusive. Assign them to rows_391_to_500.
- Select the item at row index 21 and column index 5. Assign it to row_21_column_5.</div>

## Selecting Columns and Custom Slicing ndarrays

<img alt="Dimensional Arrays" src="./images/selection_columns_updated.svg">

<img alt="Dimensional Arrays" src="./images/selection_1darray_updated.svg">

<img alt="Dimensional Arrays" src="./images/selection_2darray_updated.svg">

<div class="alert alert-block alert-info">
<b>Vaja:</b> From the taxi ndarray:
- Select every row for the columns at indexes 1, 4, and 7. Assign them to columns_1_4_7.
- Select the columns at indexes 5 to 8 inclusive for the row at index 99. Assign them to row_99_columns_5_to_8.
- Select the rows at indexes 100 to 200 inclusive for the column at index 14. Assign them to rows_100_to_200_column_14.</div>

## Vector Math

In [65]:
sums = []
for row in my_numbers:
    row_sums = row[0] + row[1]
    sums.append(row_sums)
    
print(sums)

[11, 10, 6, 21, 14]


<div class="alert alert-block alert-info">
<b>Vaja:</b> 
Use vector addition to add fare_amount and fees_amount. Assign the result to fare_and_fees.
After you have run your code, use the variable inspector below the code box to inspect the variables.</div>



<div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>The following table lists the arithmetic operators implemented in NumPy:</p>
<table>
<thead><tr>
<th>Operator</th>
<th>Equivalent ufunc</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>+</code></td>
<td><code>np.add</code></td>
<td>Addition (e.g., <code>1 + 1 = 2</code>)</td>
</tr>
<tr>
<td><code>-</code></td>
<td><code>np.subtract</code></td>
<td>Subtraction (e.g., <code>3 - 2 = 1</code>)</td>
</tr>
<tr>
<td><code>-</code></td>
<td><code>np.negative</code></td>
<td>Unary negation (e.g., <code>-2</code>)</td>
</tr>
<tr>
<td><code>*</code></td>
<td><code>np.multiply</code></td>
<td>Multiplication (e.g., <code>2 * 3 = 6</code>)</td>
</tr>
<tr>
<td><code>/</code></td>
<td><code>np.divide</code></td>
<td>Division (e.g., <code>3 / 2 = 1.5</code>)</td>
</tr>
<tr>
<td><code>//</code></td>
<td><code>np.floor_divide</code></td>
<td>Floor division (e.g., <code>3 // 2 = 1</code>)</td>
</tr>
<tr>
<td><code>**</code></td>
<td><code>np.power</code></td>
<td>Exponentiation (e.g., <code>2 ** 3 = 8</code>)</td>
</tr>
<tr>
<td><code>%</code></td>
<td><code>np.mod</code></td>
<td>Modulus/remainder (e.g., <code>9 % 4 = 1</code>)</td>
</tr>
</tbody>
</table>

</div>
</div>


## Calculating Statistics For 1D ndarrays


<p></p><center><img alt="Method syntax" src="https://s3.amazonaws.com/dq-content/289/Method_syntax.svg"></center><p></p>


<div class="alert alert-block alert-info">
<b>Vaja:</b> Use the ndarray.max() method to calculate the maximum value of trip_mph. Assign the result to mph_max.
Use the ndarray.mean() method to calculate the average value of trip_mph. Assign the result to mph_mean.</div>

<div>

<table>
<thead>
<tr>
<th>Calculation</th>
<th>Function Representation</th>
<th>Method Representation</th>
</tr>
</thead>
<tbody>
<tr>
<td>Calculate the minimum value of <code>trip_mph</code></td>
<td><code>np.min(trip_mph)</code></td>
<td><code>trip_mph.min()</code></td>
</tr>
<tr>
<td>Calculate the maximum value of <code>trip_mph</code></td>
<td><code>np.max(trip_mph)</code></td>
<td><code>trip_mph.max()</code></td>
</tr>
<tr>
<td>Calculate the <a target="_blank" href="https://en.wikipedia.org/wiki/Mean">mean average</a> value of <code>trip_mph</code></td>
<td><code>np.mean(trip_mph)</code></td>
<td><code>trip_mph.mean()</code></td>
</tr>
<tr>
<td>Calculate the <a target="_blank" href="https://en.wikipedia.org/wiki/Median">median average</a> value of <code>trip_mph</code></td>
<td><code>np.median(trip_mph)</code></td>
<td>There is no ndarray median method</td>
</tr>
</tbody>
</table>
</div>

## Calculating Statistics For 2D ndarrays

<img alt="Dimensional Arrays" src="./images/array_method_axis_none.svg">

<img alt="Dimensional Arrays" src="./images/array_method_axis_1.svg">

<img alt="Dimensional Arrays" src="./images/array_method_axis_0.svg">



<p><img alt="The axis parameter" src="https://s3.amazonaws.com/dq-content/289/axis_param.svg"></p>


## Reading CSV files with NumPy

<p>Below is information about selected columns from the data set:</p>
<ul>
<li><code>pickup_year</code>: The year of the trip.</li>
<li><code>pickup_month</code>: The month of the trip (January is <code>1</code>, December is <code>12</code>).</li>
<li><code>pickup_day</code>: The day of the month of the trip.</li>
<li><code>pickup_location_code</code>: The airport or <a target="_blank" href="https://en.wikipedia.org/wiki/Boroughs_of_New_York_City">borough</a> where the the trip started.</li>
<li><code>dropoff_location_code</code>: The airport or borough where the the trip finished.</li>
<li><code>trip_distance</code>: The distance of the trip in miles.</li>
<li><code>trip_length</code>: The length of the trip in seconds.</li>
<li><code>fare_amount</code>: The base fare of the trip, in dollars.</li>
<li><code>total_amount</code>: The total amount charged to the passenger, including all fees, tolls and tips.</li>
</ul>


## Datatypes

<div class="text_cell_render border-box-sizing rendered_html">
<table>
<thead><tr>
<th>Data type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>bool_</code></td>
<td>Boolean (True or False) stored as a byte</td>
</tr>
<tr>
<td><code>int_</code></td>
<td>Default integer type (same as C <code>long</code>; normally either <code>int64</code> or <code>int32</code>)</td>
</tr>
<tr>
<td><code>intc</code></td>
<td>Identical to C <code>int</code> (normally <code>int32</code> or <code>int64</code>)</td>
</tr>
<tr>
<td><code>intp</code></td>
<td>Integer used for indexing (same as C <code>ssize_t</code>; normally either <code>int32</code> or <code>int64</code>)</td>
</tr>
<tr>
<td><code>int8</code></td>
<td>Byte (-128 to 127)</td>
</tr>
<tr>
<td><code>int16</code></td>
<td>Integer (-32768 to 32767)</td>
</tr>
<tr>
<td><code>int32</code></td>
<td>Integer (-2147483648 to 2147483647)</td>
</tr>
<tr>
<td><code>int64</code></td>
<td>Integer (-9223372036854775808 to 9223372036854775807)</td>
</tr>
<tr>
<td><code>uint8</code></td>
<td>Unsigned integer (0 to 255)</td>
</tr>
<tr>
<td><code>uint16</code></td>
<td>Unsigned integer (0 to 65535)</td>
</tr>
<tr>
<td><code>uint32</code></td>
<td>Unsigned integer (0 to 4294967295)</td>
</tr>
<tr>
<td><code>uint64</code></td>
<td>Unsigned integer (0 to 18446744073709551615)</td>
</tr>
<tr>
<td><code>float_</code></td>
<td>Shorthand for <code>float64</code>.</td>
</tr>
<tr>
<td><code>float16</code></td>
<td>Half precision float: sign bit, 5 bits exponent, 10 bits mantissa</td>
</tr>
<tr>
<td><code>float32</code></td>
<td>Single precision float: sign bit, 8 bits exponent, 23 bits mantissa</td>
</tr>
<tr>
<td><code>float64</code></td>
<td>Double precision float: sign bit, 11 bits exponent, 52 bits mantissa</td>
</tr>
<tr>
<td><code>complex_</code></td>
<td>Shorthand for <code>complex128</code>.</td>
</tr>
<tr>
<td><code>complex64</code></td>
<td>Complex number, represented by two 32-bit floats</td>
</tr>
<tr>
<td><code>complex128</code></td>
<td>Complex number, represented by two 64-bit floats</td>
</tr>
</tbody>
</table>

</div>

## Boolean Indexing

### Boolean Arrays

<div class="alert alert-block alert-info">
Use vectorized boolean operations to:
<li> Evaluate whether the elements in array a are less than 3. Assign the result to a_bool.</li> 
<li> Evaluate whether the elements in array b are equal to "blue". Assign the result to b_bool.</li> 
<li>  Evaluate whether the elements in array c are greater than 100. Assign the result to c_bool.</li> </div>

### Boolean Indexing with 1D ndarrays

<div class="alert alert-block alert-info">
Calculate the number of rides in the taxi ndarray that are from February:
<li> Create a boolean array, february_bool, that evaluates whether the items in pickup_month are equal to 2.</li> 
<li> Use the february_bool boolean array to index pickup_month. Assign the result to february.</li> 
<li> Use the ndarray.shape attribute to find the number of items in february. Assign the result to february_rides.</li> </div>

### Boolean Indexing with 2D ndarrays

<img alt="Dimensional Arrays" src="./images/bool_dims_updated.svg">

<div class="alert alert-block alert-info">
<b>Vaja: </b>Ceate a boolean array, tip_bool, that determines which rows have values for the tip_amount column of more than 50. Use the tip_bool array to select all rows from taxi with values tip amounts of more than 50, and the columns from indexes 5 to 13 inclusive. Assign the resulting array to top_tips. </div>

## Assigning Values

### Assigning Values in ndarrays

In [2]:
a = np.array(['red','blue','black','blue','purple'])


<div class="alert alert-block alert-info">
<b>Vaja: </b>To help you practice without making changes to our original array, we have used the ndarray.copy() method to make taxi_modified, a copy of our original for these exercises.
<li> The value at column index 5 (pickup_location) of row index 28214 is incorrect. Use assignment to change this value to 1 in the taxi_modified ndarray.</li> 
<li> The first column (index 0) contains year values as four digit numbers in the format YYYY (2016, since all trips in our data set are from 2016). Use assignment to change these values to the YY format (16) in the taxi_modified ndarray.</li> 
<li> The values at column index 7 (trip_distance) of rows index 1800 and 1801 are incorrect. Use assignment to change these values in the taxi_modified ndarray to the mean value for that column.</li> </div>

### Assignment Using Boolean Arrays

<div class="alert alert-block alert-info">
<b>Vaja: </b>We again used the ndarray.copy() method to make taxi_copy, a copy of our original for this exercise.
<li> Select the fourteenth column (index 13) in taxi_copy. Assign it to a variable named total_amount.</li> 
<li> For rows where the value of total_amount is less than 0, use assignment to change the value to 0.</li> 
 </div>

<div class="alert alert-block alert-info">
<b>Vaja: </b>We have created a new copy of our taxi dataset, taxi_modified with an additional column containing the value 0 for every row. In our new column at index 15, assign the value 1 if the pickup_location_code (column index 5) corresponds to an airport location, leaving the value as 0 otherwise by performing these three operations:
<li> For rows where the value for the column index 5 is equal to 2 (JFK Airport), assign the value 1 to column index 15.</li> 
<li>For rows where the value for the column index 5 is equal to 3 (LaGuardia Airport), assign the value 1 to column index 15.</li> 
<li> For rows where the value for the column index 5 is equal to 5 (Newark Airport), assign the value 1 to column index 15.</li> </div>

## Adding Rows and Columns to ndarrays

## Computation on NumPy Arrays: Universal Functions


### The Slowness of Loops



In [71]:
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

### Introducing UFuncs (Universal functions)


## Subarrays as no-copy views



## Copying Data



## Primer: Which is the most popular airport?

## Primer: Calculating Statistics for Trips on Clean Data